# Multi‑Agent Systems with CrewAI - Interactive Notebook

This notebook provides runnable examples to accompany [`10_crewai_multi_agent.md`](10_crewai_multi_agent.md):
- Define CrewAI agents (roles, goals, backstories)
- Create tasks and assemble crews (sequential vs hierarchical)
- Integrate web/tools and simple guardrails
- Orchestrate multi‑agent research and reporting
- Add retries/timeouts and basic observability

Ensure your `.env` includes necessary keys, e.g. OPENAI_API_KEY and SERPER_API_KEY if using SerperDevTool.

## 0) Setup and Imports

In [ ]:
import os
from typing import Optional, Dict, Any

from dotenv import load_dotenv
load_dotenv()

crewai_available = True
tools_available = True
try:
    from crewai import Agent, Task, Crew, Process
except Exception as e:
    print("CrewAI not available:", e)
    crewai_available = False

try:
    from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
except Exception as e:
    print("crewai-tools not available:", e)
    tools_available = False

print("Environment ready.")

## 1) Define Tools (optional, guarded)
These tools require API keys or internet access. We guard initialization so the notebook remains runnable without them.

In [ ]:
web_search = None
web_scrape = None
serper_search = None

if tools_available:
    try:
        web_search = WebsiteSearchTool()
    except Exception as e:
        print("WebsiteSearchTool init error:", e)
    try:
        web_scrape = ScrapeWebsiteTool()
    except Exception as e:
        print("ScrapeWebsiteTool init error:", e)
    try:
        # Requires SERPER_API_KEY in .env
        serper_search = SerperDevTool()
    except Exception as e:
        print("SerperDevTool init error:", e)

print("Tools:", {
    "web_search": web_search is not None,
    "web_scrape": web_scrape is not None,
    "serper_search": serper_search is not None,
})

## 2) Sequential Crew: Research → Analysis → Write
This example passes context through three agents with specific roles and concise outputs.

In [ ]:
if crewai_available:
    # Agents
    researcher = Agent(
        role="AI Researcher",
        goal="Discover accurate, up-to-date information about the topic.",
        backstory="Methodical, cites sources, avoids speculation.",
        allow_delegation=False,
        verbose=True,
        tools=[t for t in [serper_search, web_search, web_scrape] if t is not None]
    )

    analyst = Agent(
        role="Analyst",
        goal="Synthesize research into concise, structured insights.",
        backstory="Transforms raw notes into executive-level takeaways.",
        allow_delegation=False,
        verbose=True
    )

    writer = Agent(
        role="Technical Writer",
        goal="Write a clear, well-structured report with references.",
        backstory="Senior writer focused on clarity and correctness.",
        allow_delegation=False,
        verbose=True
    )

    # Tasks
    t1 = Task(
        description=(
            "Research the topic: '{topic}'. Identify 5 key facts, include source URLs, "
            "and note any uncertainties."
        ),
        expected_output=(
            "A bulleted list of 5 facts with short citations (URL), plus a short 'uncertainties' note."
        ),
        agent=researcher
    )

    t2 = Task(
        description=(
            "Analyze the research output and extract 3 key insights for decision-makers. Each insight one sentence."
        ),
        expected_output="3 bullet-point insights with one sentence each.",
        agent=analyst
    )

    t3 = Task(
        description=(
            "Create a 300-500 word report synthesizing the insights. Add a brief references section citing the sources."
        ),
        expected_output="A markdown report with a References section.",
        agent=writer
    )

    crew = Crew(
        agents=[researcher, analyst, writer],
        tasks=[t1, t2, t3],
        process=Process.SEQUENTIAL,
        verbose=True
    )

    try:
        result = crew.kickoff(inputs={"topic": "LangGraph use-cases in production"})
        print(result)
    except Exception as e:
        print("Sequential crew run error:", e)
else:
    print("CrewAI unavailable; skipping sequential crew demo.")

## 3) Hierarchical Crew: Manager + Workers
A manager decomposes a high-level instruction into subtasks and delegates to workers.

In [ ]:
if crewai_available:
    manager = Agent(
        role="Project Manager",
        goal="Decide plan, assign work, and ensure quality.",
        backstory="Experienced coordinator who creates subtasks for the team.",
        allow_delegation=True,
        verbose=True
    )

    worker_researcher = Agent(
        role="Research Specialist",
        goal="Find reliable information; cite sources succinctly.",
        backstory="Expert in using search and verification.",
        allow_delegation=False,
        verbose=True
    )

    worker_writer = Agent(
        role="Writer",
        goal="Produce a structured summary with references.",
        backstory="Strong editor and communicator.",
        allow_delegation=False,
        verbose=True
    )

    master_task = Task(
        description=(
            "Investigate '{topic}' and produce a 400-word executive summary with 5 bullet references."
        ),
        expected_output="A 400-word summary with 5 references (URLs).",
        agent=manager
    )

    h_crew = Crew(
        agents=[manager, worker_researcher, worker_writer],
        tasks=[master_task],
        process=Process.HIERARCHICAL,
        verbose=True
    )

    try:
        res = h_crew.kickoff(inputs={"topic": "RAG best practices 2025"})
        print(res)
    except Exception as e:
        print("Hierarchical crew run error:", e)
else:
    print("CrewAI unavailable; skipping hierarchical crew demo.")

## 4) Guardrails: Simple Retry Wrapper for External Calls
Wrap tool callables or custom integrations with retry/backoff to improve robustness.

In [ ]:
import time
from functools import wraps

def with_retry(retries=3, backoff=0.5):
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            last = None
            for i in range(retries):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    last = e
                    time.sleep(backoff * (2 ** i))
            return f"retry-error: {last}"
        return wrapper
    return deco

@with_retry(retries=2, backoff=0.25)
def fragile_fetch(url: str) -> str:
    # Placeholder for a network call; simulate instability by raising intermittently in your tests
    return f"fetched: {url[:40]}..."

print(fragile_fetch("https://example.com/long-article"))

## 5) Minimal Observability for Tools
Add simple timing logs around tool functions to measure latency.

In [ ]:
def log_tool(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        t0 = time.time()
        res = fn(*args, **kwargs)
        dt = (time.time() - t0) * 1000
        print(f"[tool] {fn.__name__} took {dt:.1f} ms")
        return res
    return wrapper

logged_fetch = log_tool(fragile_fetch)
print(logged_fetch("https://example.com"))

## 6) Exercises (Scaffolds)
Implement these to deepen your practice:

A) Research + Compare
- Two researcher agents (Framework A and B) and one analyst to compare.
- Output: a comparison table of 5 criteria with source URLs.
- Constraint: each researcher must cite ≥3 unique sources.

B) RAG‑Augmented Crew
- Add a Retriever tool to answer domain‑specific questions before web search.
- Analyst merges RAG context and web snippets into final insights.

C) Hierarchical Orchestration
- Manager creates subtasks for “market size”, “competitors”, “SWOT”.
- Workers complete subtasks; manager compiles a 1‑page brief.

D) Guardrails
- URL whitelist + max page fetch count.
- If more sources are needed, ask user to confirm (human‑in‑the‑loop).

E) Reference Validator
- Post‑process links (status 200, domain whitelist). Replace failed links.
- Add a QA agent that verifies references section integrity.

## Summary
You built sequential and hierarchical crews, integrated tools with guardrails, and set up basic observability. Extend these patterns with domain‑specific tools (RAG, APIs) and stronger validation for production.